<div align="center"><h1>HPC / SENAI / Hackathon (2023.2)<br>
Laplacian Filter </h1></div>

# Team
- Antônio Horácio Magalhães
- Leonardo Rodrigues
- Orlando Mota

### Laplaciano Sequencial 2D (Mensuração de tempo + Entradas de Cargas Variadas)

In [1]:
%%writefile laplacian-2d-sequential.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <omp.h>

void populationMatrix2D(double *a, int rows, int jsta, int jend) {

   for (int i = 0; i < rows; i++)
     for (int j = jsta - 1; j < jend; j++)
            a[i + j * rows] = (i + j + 2) * 1.;  /*stored dates in column (major column)*/

}

void showMatrix(double *a, int n) {

   for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        printf("%1.2f\t", a[i + j * n]);
      }
    printf("\n");
   }
   printf("\n");
}

void showVector(int *a, int n) {

   for (int i = 0; i < n; i++)
     printf("%d\t", a[i]);

   printf("\n\n");

}

void kernel(double *a, double *c,  int m, int n, int jsta2, int jend2, int dx, int dz) {

  double sx, sz;

  for (int j = jsta2 - 1; j < jend2; j++){
     for (int i = 1; i < (m - 1); i++){
            sx = a[(i - 1) + j * n]  + a[(i + 1) + j * n]  + 2 * a[i + j * n];
            sz = a[i + (j - 1) * n]  + a[i + (j + 1) * n]  + 2 * a[i + j * n];
            c[i + j * n] = (sx/(dx*dx)) + (sz/(dz*dz));
     }
  }

}

void PARA_RANGE(int n1,int n2, int nprocs, int myid, int jsta, int jend, int *vector_return){

   int iwork1 = (n2 - n1 + 1) / nprocs;
   int iwork2 = (n2 - n1 + 1) % nprocs;

   jsta   = (myid * iwork1) + n1 + fmin((double)myid, (double)iwork2);
   jend   = jsta + iwork1 - 1;

   if (iwork2 > myid)
    jend = jend + 1;

   vector_return[0] = jsta;
   vector_return[1] = jend;

}

int main (int argc, char *argv[]) {

   int rows    = atoi(argv[1]);
   int columns = rows;
   int myid = 0;
   int nprocs = 1;
   double dx = 1, dz = 1;
   int jsta = 1, jend = 1, jsta2, jend2;
   int *vector_return = (int *) calloc (2, sizeof(int));
   double t1, t2;

   double *a  =  (double*) calloc (rows * columns, sizeof(double));
   double *c  =  (double*) calloc (rows * columns, sizeof(double));

   PARA_RANGE(1, rows, nprocs, myid, jsta, jend, vector_return);

   jsta = vector_return[0];
   jend = vector_return[1];

   jsta2 = jsta;
   jend2 = jend;

   jsta2 = 2;
   jend2 = columns - 1;

   t1 = omp_get_wtime();
     populationMatrix2D(a, rows, jsta, jend);
     kernel(a, c, rows, columns, jsta2, jend2, dx, dz);
   t2 = omp_get_wtime();

   printf("%d x %d \t%1.3f\n",rows, columns, t2-t1);

   free(a);
   free(c);

   return 0;
}

Writing laplacian-2d-sequential.c


In [2]:
!gcc laplacian-2d-sequential.c -o laplacian-2d-sequential -fopenmp -lm -O3

In [3]:
!./laplacian-2d-sequential 18432

18432 x 18432 	6.728


In [4]:
%%writefile script-2d.sh
for i in 1024 4096 6144 8192 10240 14336 18432 22528 27648 46340
do
./$1 $i
done

Writing script-2d.sh


In [5]:
!bash script-2d.sh laplacian-2d-sequential

1024 x 1024 	0.010
4096 x 4096 	0.246
6144 x 6144 	0.596
8192 x 8192 	1.246
10240 x 10240 	1.619
14336 x 14336 	3.315
18432 x 18432 	6.742
22528 x 22528 	8.275
27648 x 27648 	14.876
46340 x 46340 	23.952


---
## Aplicações Paralelas

### 1 GPU (CUDA) - PRINCIPAL

In [6]:
%%writefile laplacian-2d-gridstrideloop.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <omp.h>
#include <cuda.h>

__global__
void populationMatrix2D(float *a, int rows, int jsta, int jend) {
    int i, j;
    int idxI = blockIdx.x * blockDim.x + threadIdx.x;
    int idxJ = blockIdx.y * blockDim.y + threadIdx.y;
    int strideI = gridDim.x * blockDim.x;
    int strideJ = gridDim.y * blockDim.y;

    
    for (i = idxI; i < rows && (i >= 0 && i < rows); i += strideI) 
        for (j = idxJ; j < jend && (j >= (jsta - 1) && j < jend); j += strideJ) 
            a[i + j * rows] = (i + j + 2) * 1.0;
}

__global__
void kernel(float *a, float *c, int m, int n, int jsta2, int jend2, int dx, int dz) {
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;
    int tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    int stride_x = blockDim.x * gridDim.x;
    int stride_y = blockDim.y * gridDim.y;
     
    for (int i = 1 + tid_x; i < (m - 1); i += stride_x) 
        for (int j = jsta2 - 1 + tid_y; j < jend2; j += stride_y){
            double sx = a[(i - 1) + j * n] + a[(i + 1) + j * n] + 2 * a[i + j * n];
            double sz = a[i + (j - 1) * n] + a[i + (j + 1) * n] + 2 * a[i + j * n];
            c[i + j * n] = (sx / (dx * dx)) + (sz / (dz * dz));
        }
}

void showMatrix_all(float *a, int n) {

   for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        printf("%1.2f\t", a[i + j * n]);
      }
    printf("\n");
   }
   printf("\n");
}

void showMatrix(float *a, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            if (i == n - 2 && j == n - 2)
                printf("%lf\n", a[i + j * n]);
        }
    }
}

void showVector(int *a, int n) {
    for (int i = 0; i < n; i++)
        printf("%d\t", a[i]);

    printf("\n\n");
}

void PARA_RANGE(int n1, int n2, int nprocs, int myid, int jsta, int jend, int *vector_return) {
    int iwork1 = (n2 - n1 + 1) / nprocs;
    int iwork2 = (n2 - n1 + 1) % nprocs;

    jsta = (myid * iwork1) + n1 + fmin((double)myid, (double)iwork2);
    jend = jsta + iwork1 - 1;

    if (iwork2 > myid)
        jend = jend + 1;

    vector_return[0] = jsta;
    vector_return[1] = jend;
}

int main(int argc, char *argv[]) {
    unsigned int rows = atoi(argv[1]);
    unsigned int columns = rows;
    int myid = 0;
    int nprocs = 1;
    float dx = 1, dz = 1;
    int jsta = 1, jend = 1, jsta2, jend2;
    int *vector_return;
    double t1, t2;

    float *a;
    float *c;
    
    t1 = omp_get_wtime();
    
    cudaMallocManaged(&vector_return, 2 * sizeof(int));
    cudaMallocManaged(&a, rows * columns * sizeof(float));
    cudaMallocManaged(&c, rows * columns * sizeof(float));

    cudaMemset(a, 0, rows * columns * sizeof(float));
    cudaMemset(c, 0, rows * columns * sizeof(float));

    PARA_RANGE(1, rows, nprocs, myid, jsta, jend, vector_return);

    jsta = vector_return[0];
    jend = vector_return[1];

    jsta2 = 2;
    jend2 = columns - 1;
    
    dim3 dimBlock(1024, 1, 1);
    dim3 dimGrid((rows + dimBlock.x - 1) / dimBlock.x, (columns + dimBlock.y - 1) / dimBlock.y, 1);

    populationMatrix2D<<<dimGrid, dimBlock, 0>>>(a, rows, jsta, jend);
    kernel<<<dimGrid, dimBlock, 0>>>(a, c, rows, columns, jsta2, jend2, dx, dz);

    cudaDeviceSynchronize();

    t2 = omp_get_wtime();

    printf("%d x %d \t%.10lf\n", rows, columns, t2 - t1);

    showMatrix(c, rows);

    // Libera os recursos
    cudaFree(vector_return);
    cudaFree(a);
    cudaFree(c);

    return 0;
}


Writing laplacian-2d-gridstrideloop.cu


In [7]:
!nvcc laplacian-2d-gridstrideloop.cu -o laplacian-2d-gridstrideloop -Xcompiler -fopenmp -O3

In [27]:
!./laplacian-2d-gridstrideloop 46340

46340 x 46340 	1.6021680739
741424.000000


In [9]:
%%writefile script-2d-cuda.sh
for i in 1024 4096 6144 8192 10240 14336 18432 22528 27648 46340
do
./$1 $i
done

Writing script-2d-cuda.sh


In [10]:
!bash script-2d-cuda.sh laplacian-2d-gridstrideloop

1024 x 1024 	0.1920272168
16368.000000
4096 x 4096 	0.1983228233
65520.000000
6144 x 6144 	0.2108811345
98288.000000
8192 x 8192 	0.2315570489
131056.000000
10240 x 10240 	0.2626442984
163824.000000
14336 x 14336 	0.3371844161
229360.000000
18432 x 18432 	0.4348978531
294896.000000
22528 x 22528 	0.5547419731
360432.000000
27648 x 27648 	0.7820298597
442352.000000
46340 x 46340 	1.5969280228
741424.000000


### 2 GPU (CUDA) - NCCL 

In [11]:
%%writefile laplacian-2d-nccl.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <omp.h>
#include <cuda.h>

#include <nccl.h>
#include <cstdio>
#include <cstdlib>

void populationMatrix2D(float *a, int rows, int jsta, int jend) {

   for (int i = 0; i < rows; i++)
     for (int j = jsta - 1; j < jend; j++)
            a[i + j * rows] = (i + j + 2) * 1.;  /*stored dates in column (major column)*/

}


__global__
void kernel(float *a, float *c, int m, int n, int jsta2, int jend2, int dx, int dz) {
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;
    int tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    int stride_x = blockDim.x * gridDim.x;
    int stride_y = blockDim.y * gridDim.y;
     
    for (int i = 1 + tid_x; i < (m - 1); i += stride_x) 
        for (int j = jsta2 - 1 + tid_y; j < jend2; j += stride_y){
            double sx = a[(i - 1) + j * n] + a[(i + 1) + j * n] + 2 * a[i + j * n];
            double sz = a[i + (j - 1) * n] + a[i + (j + 1) * n] + 2 * a[i + j * n];
            c[i + j * n] = (sx / (dx * dx)) + (sz / (dz * dz));
            
            if (i == n - 2 && j == n - 2){
                printf("%lf\n", c[i + j * n]);
            }
        }
}

void showMatrix_all(float *a, int n) {

   for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        printf("%1.2f\t", a[i + j * n]);
      }
    printf("\n");
   }
   printf("\n");
}

void showMatrix(float *a, int n) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            if (i == n - 2 && j == n - 2)
                printf("%lf\n", a[i + j * n]);
        }
    }
}

void showVector(int *a, int n) {
    for (int i = 0; i < n; i++)
        printf("%d\t", a[i]);

    printf("\n\n");
}

void PARA_RANGE(int n1, int n2, int nprocs, int myid, int jsta, int jend, int *vector_return) {
    int iwork1 = (n2 - n1 + 1) / nprocs;
    int iwork2 = (n2 - n1 + 1) % nprocs;

    jsta = (myid * iwork1) + n1 + fmin((double)myid, (double)iwork2);
    jend = jsta + iwork1 - 1;

    if (iwork2 > myid)
        jend = jend + 1;

    vector_return[0] = jsta;
    vector_return[1] = jend;
}

int main(int argc, char *argv[]) {
    unsigned int rows = atoi(argv[1]);
    unsigned int columns = rows;
    int myid = 0;
    int nprocs = 1;
    float dx = 1, dz = 1;
    int jsta = 1, jend = 1, jsta2, jend2;
    int *vector_return = (int *) calloc (2, sizeof(int));
  
    PARA_RANGE(1, rows, nprocs, myid, jsta, jend, vector_return);

    jsta = vector_return[0];
    jend = vector_return[1];

    jsta2 = 2;
    jend2 = columns - 1;

    dim3 dimBlock(1024, 1, 1);
    dim3 dimGrid((rows + dimBlock.x - 1) / dimBlock.x, (columns + dimBlock.y - 1) / dimBlock.y, 1);

    // =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-= NCCL part =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

    int data_size = rows * rows;
    int nGPUs = 0;
    cudaGetDeviceCount(&nGPUs);

    int *DeviceList = (int *)malloc(nGPUs * sizeof(int));
    
    float **d_data_a,*data;
    float **d_data_c;
    d_data_a = (float**) malloc (nGPUs * sizeof(float*));
    d_data_c = (float**) malloc (nGPUs * sizeof(float*));
    data = (float*)malloc(rows * rows * sizeof(float));

    for (int i = 0; i < nGPUs; i++)
        DeviceList[i] = i;

    /* Initializing NCCL with Multiples Devices per Thread */
    ncclComm_t *comms = (ncclComm_t *)malloc(sizeof(ncclComm_t) * nGPUs);
    cudaStream_t *s = (cudaStream_t *)malloc(sizeof(cudaStream_t) * nGPUs);
    ncclCommInitAll(comms, nGPUs, DeviceList);

    populationMatrix2D(data,rows, jsta,jend);

    for (int g = 0; g < nGPUs; g++) {
        cudaSetDevice(DeviceList[g]);
        cudaStreamCreate(&s[g]);
        cudaMalloc(&d_data_a[g], rows * columns * sizeof(float));
        cudaMalloc(&d_data_c[g], rows * columns * sizeof(float));

        if(g == 0)
           cudaMemcpy(d_data_a[g], data, rows * rows * sizeof(float), cudaMemcpyHostToDevice);
    }

    ncclGroupStart();

    for (int g = 0; g < nGPUs; g++) {
        cudaSetDevice(DeviceList[g]);
        ncclBcast(d_data_a[g], rows * columns, ncclFloat, 0, comms[g], s[g]); /*Broadcasting it to all*/
        ncclBcast(d_data_c[g], rows * columns, ncclFloat, 0, comms[g], s[g]);
    }

    ncclGroupEnd();

    for (int g = 0; g < nGPUs; g++) {
        cudaSetDevice(DeviceList[g]);
        printf("\nThis is device %d\n", g);
        kernel<<<dimGrid, dimBlock>>>(d_data_a[g], d_data_c[g], rows, columns, jsta2, jend2, dx, dz);
        cudaDeviceSynchronize();
        
    }
    
    showMatrix(d_data_c[0],rows);
    
     printf("\n"); 

    for (int g = 0; g < nGPUs; g++) { /*Synchronizing CUDA Streams*/
        cudaSetDevice(DeviceList[g]);
        cudaStreamSynchronize(s[g]);
    }

    for (int g = 0; g < nGPUs; g++) { /*Destroy CUDA Streams*/
        cudaSetDevice(DeviceList[g]);
        cudaStreamDestroy(s[g]);
    }

    for (int g = 0; g < nGPUs; g++) /*Finalizing NCCL*/
        ncclCommDestroy(comms[g]);


    /*Freeing memory*/
    free(s);
    free(DeviceList);
    free(data);
    cudaFree(d_data_a);
    cudaFree(d_data_c);
    

    return 0;
}

Writing laplacian-2d-nccl.cu


In [12]:
!nvcc laplacian-2d-nccl.cu -o laplacian-2d-nccl -lnccl

laplacian-2d-nccl.cu(104): warning #177-D: variable "data_size" was declared but never referenced

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [13]:
!./laplacian-2d-nccl 80340

In [14]:
%%writefile script-2d-nccl.sh
for i in 1024 4096 6144 8192 10240 14336 18432 22528 27648 46340
do
./$1 $i
done

Writing script-2d-nccl.sh


In [15]:
!bash script-2d-nccl.sh laplacian-2d-nccl


This is device 0
16368.000000

This is device 1
16368.000000

This is device 2
16368.000000

This is device 3
16368.000000
script-2d-nccl.sh: line 1: 61973 Segmentation fault      (core dumped) ./$1 $i

This is device 0
65520.000000

This is device 1
65520.000000

This is device 2
65520.000000

This is device 3
65520.000000
script-2d-nccl.sh: line 1: 62551 Segmentation fault      (core dumped) ./$1 $i

This is device 0
98288.000000

This is device 1
98288.000000

This is device 2
98288.000000

This is device 3
98288.000000
script-2d-nccl.sh: line 1: 63125 Segmentation fault      (core dumped) ./$1 $i

This is device 0
131056.000000

This is device 1
131056.000000

This is device 2
131056.000000

This is device 3
131056.000000
^C


## Análise Experimental

### I) Validação com Valores Pequenos

#### Parâmetros Ótimos de Execução

1. CUDA = G1D B1DT1D (1024, 1, 1)

### Tempos de execução em segundos das aplicações

|  Entradas     |CUDA|
| --------------|----|
| 1024 x 1024   | 0.18   |
| 4096 x 4096   | 0.19   |
| 6144 x 6144   | 0.20   |  
| 8192 x 8192   | 0.23   |
| 10240 x 10240 | 0.26   |  
| 14336 x 14336 | 0.33
| 18432 x 18432 | 0.45   |  
| 22528 x 22528 | 0.56
| 27648 x 27648 | 0.84   |   

### Speedups

|  Entradas     |CUDA|
| --------------|----|
| 1024 x 1024   |  0,0555  |
| 4096 x 4096   |  1,2735  |
| 6144 x 6144   |  2,9852  |  
| 8192 x 8192   |  5,3913  |
| 10240 x 10240 |  6,0384  |  
| 18432 x 18432 |  10,000  |  
| 22528 x 22528 | 26,358 |
| 27648 x 27648 |  17,572 |  

### II) Análise de Desempenho - `46340`

#### Parâmetros Ótimos de Execução

1. CUDA = G1D B1DT1D (1024, 1, 1)

### Tempo de execução em segundos das aplicações

|  Entradas    | Sequencial | CUDA
| -------------| ---------- | ----
| 46340 x 46340|   29.324   | 1.49


### Speedup

|  Senha          | CUDA
|-----------------| ----
| 46340 x 46340   |  19.68X


## Conclusões

Among all possible applications to improve code performance, we decided to focus on CUDA, as it is the tool with the best ease of use and availability of speedup among those presented. We were very successful in using CUDA on a single GPU, where we achieved a speedup of approximately 20x. However, when we attempted to further improve performance by adopting a multi-GPU strategy using NCCL, we encountered issues in the execution of the final code. Therefore, we decided to concentrate on working with CUDA on a single GPU to extract the best possible results from it.

## Referências Biliográficas

* G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

* S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

* David A. Patterson and John L. Hennessy. Computer Organization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

* An Introduction to Parallel Programming by Peter S. Pacheco. Morgan Kauffman.

* W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabribridge U. Press, 1994.